In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

2023-08-25 18:00:17.284796: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df=pd.read_csv("preprocessed_isro_data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186298 entries, 0 to 186297
Data columns (total 22 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Datetime                        186298 non-null  object 
 1   DTG-1_PCH_ANALOG_RAT (Analog)   186298 non-null  float64
 2   DTG-1_ROL_ANALOG_RAT (Analog)   186298 non-null  float64
 3   DTG-1_PCH_FINE_RATE (Analog)    186298 non-null  float64
 4   DTG-1_ROL_FINE_RATE (Analog)    186298 non-null  float64
 5   DTG-2_YAW_ANALOG_RAT (Analog)   186298 non-null  float64
 6   DTG-2_PCH_ANALOG_RAT (Analog)   186298 non-null  float64
 7   DTG-2_YAW_FINE_RATE (Analog)    186298 non-null  float64
 8   DTG-2_PCH_FINE_RATE (Analog)    186298 non-null  float64
 9   DTG-1_TH_TEMP (Analog)          186298 non-null  float64
 10  DTG-1_ELECTRONICS_TH (Analog)   186298 non-null  float64
 11  DTG-2_TH_TEMP (Analog)          186298 non-null  float64
 12  DTG-2_ELECTRONIC

In [3]:
df['Datetime'] = pd.to_datetime(df['Datetime'])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186298 entries, 0 to 186297
Data columns (total 22 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   Datetime                        186298 non-null  datetime64[ns]
 1   DTG-1_PCH_ANALOG_RAT (Analog)   186298 non-null  float64       
 2   DTG-1_ROL_ANALOG_RAT (Analog)   186298 non-null  float64       
 3   DTG-1_PCH_FINE_RATE (Analog)    186298 non-null  float64       
 4   DTG-1_ROL_FINE_RATE (Analog)    186298 non-null  float64       
 5   DTG-2_YAW_ANALOG_RAT (Analog)   186298 non-null  float64       
 6   DTG-2_PCH_ANALOG_RAT (Analog)   186298 non-null  float64       
 7   DTG-2_YAW_FINE_RATE (Analog)    186298 non-null  float64       
 8   DTG-2_PCH_FINE_RATE (Analog)    186298 non-null  float64       
 9   DTG-1_TH_TEMP (Analog)          186298 non-null  float64       
 10  DTG-1_ELECTRONICS_TH (Analog)   186298 non-null  float64

In [96]:
from tensorflow import keras

datetime_series = df['Datetime']
features = df.iloc[:, 1:].values 

input_dim = features.shape[1]  
encoding_dim = 5 

encoder = keras.Sequential([
    keras.layers.Input(shape=(input_dim,)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(encoding_dim, activation='relu')
])


decoder = keras.Sequential([
    keras.layers.Input(shape=(encoding_dim,)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(input_dim, activation='relu') 
])

autoencoder = keras.Sequential([encoder, decoder])

autoencoder.compile(optimizer='adam', loss='mse')

autoencoder.fit(features, features, epochs=10, batch_size=32)

encoded_data = encoder.predict(features)
decoded_data = decoder.predict(encoded_data)

Epoch 1/10
5822/5822 [==============================] - 18s 3ms/step - loss: 0.2486
Epoch 2/10
5822/5822 [==============================] - 13s 2ms/step - loss: 0.0857
Epoch 3/10
5822/5822 [==============================] - 14s 2ms/step - loss: 0.0563
Epoch 4/10
5822/5822 [==============================] - 13s 2ms/step - loss: 0.0472
Epoch 5/10
5822/5822 [==============================] - 13s 2ms/step - loss: 0.0472
Epoch 6/10
5822/5822 [==============================] - 13s 2ms/step - loss: 0.0472
Epoch 7/10
5822/5822 [==============================] - 13s 2ms/step - loss: 0.0472
Epoch 8/10
5822/5822 [==============================] - 13s 2ms/step - loss: 0.0203
Epoch 9/10
5822/5822 [==============================] - 14s 2ms/step - loss: 1.9430e-04
Epoch 10/10
5822/5822 [==============================] - 9s 1ms/step


In [98]:
reconstruction_errors = np.mean(np.square(features - decoded_data), axis=1)
print(reconstruction_errors)
print(max(reconstruction_errors))
print(sorted(reconstruction_errors, reverse= True)[:10])

threshold = max(reconstruction_errors)

first_anomaly_index = np.argmax(reconstruction_errors >= threshold)

first_anomaly_datetime = datetime_series.iloc[first_anomaly_index]

print("Datetime of the first anomaly:", first_anomaly_datetime)

[1.13125770e-04 1.17660998e-04 1.17660998e-04 ... 6.98836962e-05
 6.98838186e-05 6.98838186e-05]
0.10715991301959006
[0.10715991301959006, 0.10016686307262068, 0.10016686307262068, 0.10016686307262068, 0.0953177798518943, 0.0953177798518943, 0.09528942329039782, 0.09525049725823169, 0.09522297284812156, 0.09522297284812156]
Datetime of the first anomaly: 2018-05-10 10:48:01.660014
